In [6]:
!pip install opencv-python
!pip install tensorflow
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install tqdm
!pip install ultralytics


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import random
import os
import shutil
import cv2
import glob
import torch
import torchvision
import numpy as np
import json
import yaml
import time
from tqdm import tqdm
from PIL import Image
from torchvision import datasets, models, transforms
from ultralytics import YOLO

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# data resize/padding
data_root = 'D:\\Falldown\\dataset'
train_root = f'{data_root}\\train\\images'
val_root = f'{data_root}\\val\\images'
test_root = f'{data_root}\\test\\images'

# Helper function to resize and pad images
def letterbox_image(image, target_size=(640, 640)):
    # 원본 이미지 크기
    h, w = image.shape[:2]
    target_w, target_h = target_size

    # 비율 계산
    scale = min(target_w / w, target_h / h)

    # 새로운 이미지 크기
    new_w = int(w * scale)
    new_h = int(h * scale)

    # 이미지 리사이즈
    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    # 패딩 적용
    pad_w = (target_w - new_w) // 2
    pad_h = (target_h - new_h) // 2

    # 이미지를 타겟 크기로 채워서 새로운 이미지를 만듦
    padded_image = cv2.copyMakeBorder(resized_image, pad_h, target_h - new_h - pad_h, pad_w, target_w - new_w - pad_w,
                                      cv2.BORDER_CONSTANT, value=[128, 128, 128])

    return padded_image

# Process Non-Fall Data with tqdm
for root in [train_root, val_root, test_root]:
    image_files = [file for file in os.listdir(root) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    # tqdm 적용
    for file in tqdm(image_files, desc=f"Processing images in {root}", unit="file"):
        file_path = os.path.join(root, file)
        
        # 이미지 읽기
        image = cv2.imread(file_path)
        if image is None:  # 이미지 읽기 실패 시 np.fromfile과 cv2.imdecode 사용
            try:
                image = cv2.imdecode(np.fromfile(file_path, np.uint8), cv2.IMREAD_COLOR)
            except Exception as e:
                print(f"Error reading image {file_path}: {e}")
                continue
        
        if image is not None:
            resized_image = letterbox_image(image)
            output_path = os.path.join(root, file)
            cv2.imwrite(output_path, resized_image)
        else:
            print(f"Warning: Could not process image file {file_path}")

Processing images in D:\Falldown\Dataset\Train\Image: 100%|███████████████████| 99893/99893 [16:28<00:00, 101.02file/s]
Processing images in D:\Falldown\Dataset\Val\Image: 100%|█████████████████████| 11360/11360 [01:50<00:00, 103.24file/s]
Processing images in D:\Falldown\Dataset\Test\Image: 100%|█████████████████████| 45280/45280 [31:53<00:00, 23.66file/s]


In [15]:
# JSON 라벨에 있는 bbox 정보를 YOLO 라벨 형식으로 변환 (정규화 포함)
train_json = 'D:\\Falldown\\dataset\\train\\json'
val_json = 'D:\\Falldown\\dataset\\val\\json'
test_json = 'D:\\Falldown\\dataset\\test\\json'

def convert_bbox(bbox, img_width, img_height):
    x1, y1, x2, y2 = map(float, bbox.split(','))
    
    width = (x2 - x1) / img_width
    height = (y2 - y1) / img_height
    center_x = (x1 + x2) / (2 * img_width)
    center_y = (y1 + y2) / (2 * img_height)
    
    return center_x, center_y, width, height

def determine_class(file_name):
    file_name_lower = file_name.lower()
    if 'by' in file_name_lower or 'sy' in file_name_lower or 'fy' in file_name_lower:
        return 1
    else:
        return 0

def process_json(json_path, output_dir, original_size=(3840, 2160), new_size=(640, 640)):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    orig_width, orig_height = original_size
    new_width, new_height = new_size

    # 스케일 및 패딩 계산
    scale = min(new_width / orig_width, new_height / orig_height)
    scaled_width = int(orig_width * scale)
    scaled_height = int(orig_height * scale)
    pad_x = (new_width - scaled_width) // 2
    pad_y = (new_height - scaled_height) // 2

    # JSON에서 bbox 정보를 가져와 YOLO 형식으로 변환
    bbox = data['bboxdata']['bbox_location']
    x, y, w, h = convert_bbox(bbox, orig_width, orig_height)

    # 스케일과 패딩을 적용한 최종 좌표 계산
    x = (x * scaled_width + pad_x) / new_width
    y = (y * scaled_height + pad_y) / new_height
    w = (w * scaled_width) / new_width
    h = (h * scaled_height) / new_height

    # 클래스 ID 결정
    class_id = determine_class(data['metadata']['file_name'])
    
    # YOLO 형식으로 저장
    result = f"{class_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f}"
    
    output_filename = os.path.splitext(data['metadata']['file_name'])[0] + '.txt'
    output_path = os.path.join(output_dir, output_filename)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(result)

def process_all_json_in_folder(input_folder, output_folder, original_size=(3840, 2160), new_size=(640, 640)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.endswith('.json'):
            json_path = os.path.join(input_folder, filename)
            process_json(json_path, output_folder, original_size, new_size)
    
    print(f"Processed all JSON files to yolo_labels in {input_folder}")

# 각 데이터셋 처리
datasets = [
    ('Train', train_json),
    ('Val', val_json),
    ('Test', test_json)
]

for dataset_name, json_folder in datasets:
    output_folder = os.path.join(os.path.dirname(json_folder), 'labels')
    print(f"Processing {dataset_name} dataset...")
    process_all_json_in_folder(json_folder, output_folder)

print("All datasets processed.")

Processing Train dataset...
Processed all JSON files to yolo_labels in D:\Falldown\Dataset\Train\Label
Processing Val dataset...
Processed all JSON files to yolo_labels in D:\Falldown\Dataset\Val\Label
Processing Test dataset...
Processed all JSON files to yolo_labels in D:\Falldown\Dataset\Test\Label
All datasets processed.


In [9]:
data_root = 'D:\\Falldown\\dataset'
train_root = f'{data_root}\\train\\images'
val_root = f'{data_root}\\val\\images'
class_names = {0 : 'Non_Fall', 1 : 'Fall'}
num_classes = len(class_names)

yaml_info = {
    'path' : data_root,
    'names': class_names,
    'nc': num_classes,
    'train': train_root,
    'val': val_root
}

with open('yaml_info_yolov8s.yaml', 'w') as f : 
    yaml.dump(yaml_info, f)
print(f'이 경로에 yaml파일 생성 : {data_root}')

이 경로에 yaml파일 생성 : D:\Falldown\dataset


In [15]:
model = YOLO('yolov8s.pt')

In [ ]:
start_time = time.time()

result = model.train(data = 'D:\\project\\prjvenv\\yaml_info_yolov8s.yaml', epochs = 50, batch = 16, imgsz =640, device = device, workers = 20, amp = True, patience = 30, name = 'human_fall_s')

end_time = time.time()
execution_time = end_time - start_time
print(f"실행 시간: {execution_time:.4f} 초") # 약 20시간 소요

In [19]:
# 평가 시작 시간 기록
start_time = time.time()

# YOLO 모델 로드 및 검증 실행
model = YOLO('D:\\Falldown\\runs\\detect\\human_fall_s30\\weights\\best.pt')
results = model.val(data='D:\\Falldown\\yaml_info_yolov8s.yaml', imgsz=640, batch=32, device="cuda")

# 평가 종료 시간 기록
end_time = time.time()
execution_time = end_time - start_time

# 결과 출력
print(f"Validation Results:")
print(f"mAP50: {results.box.map50:.4f}")
print(f"mAP50-95: {results.box.map:.4f}")
print(f"실행 시간: {execution_time:.4f} 초")

Ultralytics 8.3.23  Python-3.10.11 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning D:\Falldown\dataset\val\labels.cache... 11360 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11360/11
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 355/355 [01:


                   all      11360      11360      0.745      0.855      0.818      0.796
              Non_Fall       5680       5680      0.745      0.911      0.782      0.764
                  Fall       5680       5680      0.746      0.799      0.854      0.827
Speed: 0.2ms preprocess, 5.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\val12
Validation Results:
mAP50: 0.8183
mAP50-95: 0.7957
실행 시간: 285.5395 초
